# Neuroscience 🧠 using `fastplotlib` 🦜 and `pynapple` 🍍

This notebook will build up a complex visualization using `fastplotlib`, in conjunction with `pynapple`, to show how `fastplotlib` can be a powerful tool in analysis and visualization of neural data!

In [1]:
import fastplotlib as fpl
import pynapple as nap
import numpy as np
from ipywidgets import IntSlider, Layout, VBox, HBox, FloatSlider
from skimage import measure
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from scipy.ndimage import gaussian_filter1d
from sidecar import Sidecar
from time_store import TimeStore

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01,\x00\x00\x007\x08\x06\x00\x00\x00\xb6\x1bw\x99\x…

MESA-INTEL: warning: cannot initialize blitter engine
Detected skylake derivative running on mesa i915. Clears to srgb textures will use manual shader clears.
Detected skylake derivative running on mesa i915. Clears to srgb textures will use manual shader clears.
Detected skylake derivative running on mesa i915. Clears to srgb textures will use manual shader clears.
Detected skylake derivative running on mesa i915. Clears to srgb textures will use manual shader clears.
Detected skylake derivative running on mesa i915. Clears to srgb textures will use manual shader clears.


Available devices:
🯄 (default) | Intel(R) Arc(tm) Graphics (MTL) | IntegratedGPU | Vulkan | Mesa 24.0.8-1
❗ | llvmpipe (LLVM 17.0.6, 256 bits) | CPU | Vulkan | Mesa 24.0.8-1 (LLVM 17.0.6)
❗ | Mesa Intel(R) Arc(tm) Graphics (MTL) | IntegratedGPU | OpenGL | 


In [2]:
import warnings
warnings.simplefilter('ignore')
fpl.config.party_parrot = True

## Load the data 

#### Recording of a freely-moving mouse imaged with a Miniscope (1-photon imaging). The area recorded is the postsubiculum - a region that is known to contain head-direction cells, or cells that fire when the animal's head is pointing in a specific direction. 

In [3]:
data = nap.load_file("./data.nwb")

In [4]:
data

data
┍━━━━━━━━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━┑
│ Keys                  │ Type        │
┝━━━━━━━━━━━━━━━━━━━━━━━┿━━━━━━━━━━━━━┥
│ position_time_support │ IntervalSet │
│ RoiResponseSeries     │ TsdFrame    │
│ calcium_video         │ TsdTensor   │
│ beh_video             │ TsdTensor   │
│ z                     │ Tsd         │
│ y                     │ Tsd         │
│ x                     │ Tsd         │
│ rz                    │ Tsd         │
│ ry                    │ Tsd         │
│ rx                    │ Tsd         │
┕━━━━━━━━━━━━━━━━━━━━━━━┷━━━━━━━━━━━━━┙

### Let's view the behavior and calcium data

**NOTE:** We are going to be using a WIP `TimeStore` model to help synchronize our visualization in time. Hopefully, by the end of the summer we will have developed a tool ([`pynaviz`](https://github.com/pynapple-org/pynaviz)) that makes these visualizations and synchronizations even easier :D

In [5]:
time_store = TimeStore()

Behavior data and shape 🐭

In [6]:
behavior_data = data["beh_video"]
behavior_data.shape  # (time, x, y)

(9045, 204, 256)

Calcium data and the shape 🔬

In [7]:
calcium_data = data["calcium_video"]
calcium_data.shape  # (time, x, y)

(17886, 136, 166)

The behavior tracks data needs to be scaled 

In [8]:
print(data["x"].min(), data["x"].max())
print(data["z"].min(), data["z"].max())

-0.460045 0.446303
-0.369381 0.537622


Scale the behavior tracks data w.r.t. the behavior movie dims

In [9]:
data["x"] = data["x"] + np.abs(data["x"].min())
data["x"] = data["x"] * behavior_data.shape[1]

data["z"] = data["z"] + np.abs(data["z"].min())
data["z"] = data["z"] * behavior_data.shape[2]

Array of the behavior tracks

In [10]:
tracks_data = np.column_stack([data["z"], data["x"]])

In [11]:
tracks_data 

Time (s)          0        1
----------  -------  -------
7.39305     57.6402  49.6919
7.4014      57.9259  49.7264
7.40975     58.1739  49.7525
7.41805     58.3542  49.7605
7.4264      58.538   49.7691
7.43475     58.7461  49.7719
7.44305     58.9729  49.7568
...
1213.17765  95.9529  96.8916
1213.186    95.8935  96.9412
1213.19435  95.7737  96.9828
1213.20265  95.6608  97.0346
1213.211    95.5814  97.0842
1213.21935  95.5848  97.0591
1213.22765  95.6273  97.0328
dtype: float64, shape: (144709, 2)

#### Set our view of the data to where both behavior and position data are available:

In [12]:
behavior_data = behavior_data.restrict(data["position_time_support"])
calcium_data = calcium_data.restrict(data["position_time_support"])

(data["position_time_support"].start[0], data["position_time_support"].end[0])

(7.39305, 1213.22765)

In [13]:
# calculate min frame across movie
# remove vignette effect from 1p endoscopic imaging
min_frame = calcium_data.min(axis=0)

# just to show you what this looks like
iw = fpl.ImageWidget(min_frame)
iw.show()

RFBOutputContext()

Detected skylake derivative running on mesa i915. Clears to srgb textures will use manual shader clears.


JupyterOutputContext(children=(JupyterWgpuCanvas(), IpywidgetToolBar(children=(Button(icon='expand-arrows-alt'…

In [14]:
iw.close()

## Create a big viz for calcium and behavior video! 🎨

In [15]:
# make figure, calcium on left, behavior on right
nap_figure = fpl.Figure(shape=(1,2), names=[["calcium", "behavior"]])

# image graphic to display current calcium frame
calcium_graphic = nap_figure["calcium"].add_image(data=calcium_data[0] - min_frame, name="calcium_frame", cmap="gnuplot2")

# a UI tool to help set and visualize vmin-vmax
hlut = fpl.widgets.histogram_lut.HistogramLUT(data=calcium_data, image_graphic=calcium_graphic)
# add this to the right dock
nap_figure["calcium"].docks["right"].add_graphic(hlut)
nap_figure["calcium"].docks["right"].size = 80
nap_figure["calcium"].docks["right"].auto_scale(maintain_aspect=False)
nap_figure["calcium"].docks["right"].controller.enabled = False

# image graphic to display current behavior video frame
behavior_graphic = nap_figure["behavior"].add_image(data=behavior_data[0], cmap="gray")

# line to display the behavior tracks
tracks_graphic = nap_figure["behavior"].add_line(tracks_data, cmap="winter", thickness=2, alpha=0.5, offset=(12, 0, 2))

RFBOutputContext()

#### Create a slider that updates the behavior and calcium videos using `pynapple`

In [16]:
# This time will be in seconds
synced_time = FloatSlider(min=data["position_time_support"].start, max=data["position_time_support"].end, step=0.01, description="s")

# auto-resize slider
@nap_figure.renderer.add_event_handler("resize")
def resize_slider(ev):
    synced_time.layout = Layout(width=f"{ev.width}px")

#### Add the components of our visualization to the `TimeStore` model to be synchronized 🕰️

In [17]:
# add the slider
time_store.subscribe(subscriber=synced_time)

def substract_min(frame):
    """subtract min frame from current frame"""
    global min_frame

    return frame - min_frame

# add our calcium data
time_store.subscribe(subscriber=calcium_graphic, data=calcium_data, data_filter=substract_min)

# add our behavior data
time_store.subscribe(subscriber=behavior_graphic, data=behavior_data)

**Here we are going to use `sidecar` to organize our visualizations better :D**

In [18]:
sc = Sidecar()
with sc:
    display(VBox([nap_figure.show(), synced_time]))

### Visualize head angle just by setting the cmap transform, it's that simple! 🪄

In [19]:
# set cmap transform from "ry" head angle
tracks_graphic.cmap.transform = data["ry"].to_numpy()

# change to a heatmap more suitable for this data
tracks_graphic.cmap = "hsv"

## Visualize other kinematics just by setting the cmap transform! :D 

In [20]:
def get_velocity(array):
    return np.gradient(np.abs(gaussian_filter1d(array, sigma=10)))

In [21]:
tracks_graphic.cmap.transform = get_velocity(data["z"].to_numpy())
tracks_graphic.cmap = "seismic"  # diverging colormap, velocities are negative and positive

In [22]:
tracks_graphic.cmap.transform = get_velocity(data["x"].to_numpy())

Let's go back to head direction

In [23]:
tracks_graphic.cmap.transform = data["ry"].to_numpy()
tracks_graphic.cmap = "hsv"

# Visualize Calcium Imaging ROIs

#### Calculate the spatial contours and overlay them on the raw calcium data

In [24]:
# get the masks
contour_masks = data.nwb.processing['ophys']['ImageSegmentation']['PlaneSegmentation']['image_mask'].data[:]
# reshape the masks into a list of 105 components
contour_masks = list(contour_masks.reshape((len(contour_masks), 166, 136)))

In [25]:
# calculate each contour from the mask using `scikit-image.measure`
contours = list()

for mask in contour_masks:
    contours.append(np.vstack(measure.find_contours(mask)))

#### Add the calculated contours as an overlay to the calcium video

In [26]:
contours_graphic = nap_figure["calcium"].add_line_collection(data=contours, colors="w", alpha=0.8)

**It is very easy to see that many of the identified neurons may be "bad" candidates. Let's remove them from the dataset as we go on in our anaylsis.**

### Select only head-direction neurons

In [27]:
# get the temporal data (calcium transients) from the nwb notebook
temporal_data = data["RoiResponseSeries"][:].restrict(data["position_time_support"])
temporal_data

Time (s)           0        1        2        3        4  ...
-----------  -------  -------  -------  -------  -------  -----
7.4          4.25336  0.1776   1.36078  0.4559   0.22819  ...
7.433333     4.23919  0.17688  1.35602  0.45423  0.22733  ...
7.466667     4.22506  0.17617  1.35128  0.45257  0.22647  ...
7.5          4.21097  0.17546  1.34655  0.5235   0.22562  ...
7.533333     4.19693  0.17475  1.34184  0.57631  0.22476  ...
7.566667     4.18294  0.17404  1.33714  0.74999  0.22392  ...
7.6          4.16899  0.17334  1.33246  0.88075  0.22307  ...
...
1192.166667  2.54202  0.14531  0.44013  0.5681   0.65477  ...
1192.2       2.53029  0.14775  0.43842  0.56657  0.65227  ...
1192.233333  2.51861  0.14962  0.43671  0.56505  0.64979  ...
1192.266667  2.50698  0.15104  0.435    0.56354  0.64731  ...
1192.3       2.49541  0.15209  0.43331  0.56202  0.64485  ...
1192.333333  2.48389  0.15283  0.43162  0.58476  0.64239  ...
1192.366667  2.47242  0.15333  0.42994  0.62802  0.63994  ...
dt

In [28]:
# compute 1D tuning curved based on head angle
head_angle = data["ry"]

tuning_curves = nap.compute_1d_tuning_curves_continuous(temporal_data, head_angle, nb_bins = 120)

#### Select the top 50 components 

In [29]:
# select good components 
good_ixs = list(np.argsort(np.ptp(tuning_curves, axis=0))[-50:])
bad_ixs = list(np.argsort(np.ptp(tuning_curves, axis=0))[:-50])

#### Color the "good" and "bad" components

In [30]:
contours_graphic[good_ixs].colors = "w"
contours_graphic[bad_ixs].colors = "magenta"

#### Sort the "good" components based on preferred head direction

In [31]:
# sorting the "good" neurons based on preferred directions
sorted_ixs = tuning_curves.iloc[:,good_ixs].idxmax().sort_values().index.values

In [32]:
sorted_ixs

array([75, 34, 77, 86, 21, 16,  6,  4, 58, 44, 14, 33, 94, 98, 90, 76,  7,
        5, 82, 28, 15, 88, 45, 39,  0,  8, 20, 13, 24, 60, 18, 27, 10, 78,
        2, 85,  3, 19, 38, 17, 30, 29, 25, 84, 12, 26, 41,  9, 11,  1])

#### Filter the dataset to only use the sorted "good" components

In the rest of the demo we will only be using the sub-sampled components.

In [33]:
# filter dataset based on sortex indices
temporal_data = temporal_data[:,sorted_ixs]
contours = [contours[i] for i in sorted_ixs]

#### Plot only the "good" components

In [34]:
# remove the graphic of all the components
nap_figure["calcium"].remove_graphic(contours_graphic)

# re-plot only the good ixs
contours_graphic = nap_figure[0, 0].add_line_collection(data=contours, colors="w", alpha=0.8)

## Visualize all calcium tracing using an `ImageGraphic` to display a Heatmap

In [35]:
# create a figure, 2 rows, 1 column
temporal_fig = fpl.Figure(shape=(2,1), names=[["temporal-heatmap"], ["tuning-curve"]])

RFBOutputContext()

In [36]:
# we need to transpose our temporal data so that it is (# components, time (s))
raw_temporal = temporal_data.to_numpy().T

# use 'hsv' colormap to represent preferred head direction 
heatmap_graphic = temporal_fig[0,0].add_image(data=raw_temporal, cmap="plasma", name="traces")

#### Add a `LinearSelector` that we can map to our behavior and calcium videos

In [37]:
time_selector = heatmap_graphic.add_linear_selector()

In [38]:
component_selector = heatmap_graphic.add_linear_selector(axis="y")

In [39]:
# subscribe selector to timestore
time_store.subscribe(subscriber=time_selector, multiplier=temporal_data.rate)

#### Let's view everything together

In [40]:
@nap_figure.renderer.add_event_handler("resize")
def resize_temporal_fig(ev):
    temporal_fig.canvas.set_logical_size(ev.width, 300)

sc = Sidecar()

with sc:
    display(VBox([nap_figure.show(), temporal_fig.show(maintain_aspect=False), synced_time]))

In [41]:
# select the first component
ix = 0

# set the first component colors to magenta
contours_graphic[ix].colors = "green"

# get the tuning curve of the first component 
tuning_ix = sorted_ixs[ix]

tuning_curve = tuning_curves.T.iloc[tuning_ix]

# add the tuning curve to the plot as a line
tuning_graphic = temporal_fig["tuning-curve"].add_line(data=tuning_curve, offset=(0,0,0))
temporal_fig["tuning-curve"].auto_scale(maintain_aspect=False)

### Add an event handler that allows us to "scroll" through the traces and tuning curves :D

In [42]:
# add an event handler that allows tabbing up and down traces
@component_selector.add_event_handler("selection")
def update_selected_trace(ev):
    ix = ev.get_selected_index()
    
    # reset the colors of the components to white
    contours_graphic.colors = "w"

    # set the selected component colors to magenta
    contours_graphic[ix].colors = "green"

    nap_figure["calcium"].camera.show_object(contours_graphic[ix].world_object)

    # get tuning curve of the selected component
    tuning_ix = sorted_ixs[ix]

    tuning_curve = tuning_curves.T.iloc[tuning_ix]

    # remove the current tuning curve add the new one
    #global tuning_graphic
    temporal_fig["tuning-curve"].graphics[0].data[:,1] = tuning_curve
    temporal_fig["tuning-curve"].auto_scale(maintain_aspect=False)
    

    #tuning_graphic = tstack_fig[1,0].add_line(data=tuning_curve, offset=(0,0,0))

# Downstream analysis, view a PCA of the calcium

In [43]:
from sklearn.decomposition import PCA
from scipy.stats import zscore

#### Perform PCA

In [44]:
pca = PCA(n_components=3)

zscored = zscore(np.sqrt(temporal_data.to_numpy()), axis=1)
calcium_pca = pca.fit_transform(gaussian_filter1d(zscored, sigma=3))

#### Plot the PCA results

To get a proper colormap transform based on the head angle data, need to interpolate the time scale.

In [45]:
# restrict the head angle data 
ry_restrict = data["ry"].restrict(data["position_time_support"])

In [46]:
x = np.arange(0, temporal_data.shape[0])
xp = np.linspace(0, temporal_data.shape[0], ry_restrict.shape[0])

# interpolate 
ry_transform =  np.interp(x, xp, fp=ry_restrict)  # use the y-values

In [47]:
fig_pca = fpl.Figure(
    cameras="3d",
    controller_types="orbit",
)
fig_pca[0, 0].add_scatter(calcium_pca, cmap="hsv", cmap_transform=ry_transform, sizes=4, alpha=0.4)
marker_graphic = fig_pca[0, 0].add_scatter(calcium_pca[0], sizes=10)

fig_pca.show()

RFBOutputContext()

JupyterOutputContext(children=(JupyterWgpuCanvas(), IpywidgetToolBar(children=(Button(icon='expand-arrows-alt'…

#### Subscribe the PCA marker to the `TimeStore` model

In [48]:
# create a `pynapple.TsdFrame` for the PCA data
pca_data = nap.TsdFrame(t=temporal_data.t, d=calcium_pca)

time_store.subscribe(subscriber=marker_graphic, data=pca_data, multiplier=temporal_data.rate)

Can change the controller

In [ ]:
fig_pca[0, 0].controller = "fly"

In [57]:
nap_figure["behavior"].graphics[1].thickness = 0.1